In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import sys
sys.path.append('/solver/')
import solver

from solver.Mesh import *
from solver.SolverField import *
from solver.aux_solver import *
from solver.SbnSolver import *
from solver.DigitalTwin import *

# 1. Simulation of 2 laser beams propagating inside an SBN crystal

In [2]:
### arrayfire configuration
print('Available Backends -> ', af.get_available_backends())

###set backend and device
af.set_backend('cuda')
af.set_device(0)

###print info
print(af.info_str())


Available Backends ->  ('cuda', 'opencl')
ArrayFire v3.6.1 (CUDA 64bit)
[0] : GeForce_GTX_1050 (Compute 6.1)



## 1.1. Some Auxiliary Functions

In [3]:
def phase_slip(x,y,A,w,x0,y0,vx,vy,width):
    new_field = A*np.exp(-(2*((x - x0)*(x - x0)+ (y - y0)*(y - y0)) / (w*w))**4) *np.exp((
                         ((x-x0)<0)*1j*np.pi))
    
    return new_field

def super_gauss(x,y,A,w,x0,y0,vx,vy,width,phase=0,x00=0,y00=0):
    aux = (((x - x0)*(x - x0)+ (y - y0)*(y - y0)) <= ( 0.8**2* (w*w)))
    new_field = A*np.exp(-(2*((x - x0)*(x - x0)+ (y - y0)*(y - y0)) / (w*w))**2)+1e-6
    new_field = new_field * (np.exp((1j*((vx*(x-x00))+(vy*(y-y00)))*aux)+1j*phase))
    return new_field

def whitenoise_2d_field_1(x,A):
    
    new_field = A*(0.5-np.random.rand(x.shape[0],x.shape[1]))*2         
    return new_field

def whitenoise_2d_field(x,A):
    np.random.seed()
    new_field = np.random.normal(0,scale=A, size=(x.shape[0],x.shape[1]))#
    return new_field

def whitenoise_1d_field(x,A):
    new_field = np.random.normal(0,scale=A, size=(x.shape[0]))#
    return new_field


def periodic_noise(y,A,k):
    new_field = A*np.sin(k*y)    
    return new_field

## 1.2. Configure Simulator and run

$$ i \partial_z E_{1} + \frac{1}{2} \nabla ^{2}_{\perp} E_{1} + c_{1} \Delta n_{max} \frac{|E_1|^2 + |E_2|^2}{I_{sat}+|E_1|^2 + |E_2|^2} E_1+ \alpha E_1 = 0$$
$$ i \partial_z E_{2} + \frac{1}{2\gamma} \nabla ^{2}_{\perp} E_{2} + c_{2} \Delta n_{max} \gamma \frac{|E_1|^2 + |E_2|^2}{I_{sat}+|E_1|^2 + |E_2|^2} E_2+ \alpha E_2 = 0$$

![image](image.png)

$\Delta n_{max} = \frac{1}{2}n_{e}^{3}V/d$ with $d=5mm$ being the crystal width. The typical crystal length is around 2cm.

## 2. Setting up a 2D simulation with a vortex state

In [33]:
for run_n in range(0,100):

    # Simulation Box to match experiment
    pixel_pitch = 1.25e-6
    magnification = 200/60
    x_camera = np.arange(0,4000)*pixel_pitch  #micrometers
    y_camera = np.arange(0,3000)*pixel_pitch



    ## Crystal
    lx=x_camera[-1]/ magnification
    ly=y_camera[-1]/ magnification
    lz=20e-3
    n_passages = 2 ###increase to see more dynamics, but in reality we have just 1

    ## start digital twin model
    c1 = 40/250 #rxx / r33
    c2 = 1#40/250 # rxx / r33
    V=400
    Isat = 200 #mW/cm2

    SBN_sim = DigitalTwin(lx=lx,ly=ly,lz=n_passages * lz,
                        V=V,c1=c1,c2=c2,Isat=Isat,
                        alpha=0)
    run_number=run_n
    saveDir = 'D:\\Quarmen2d_sim_new_sims_2passages_256\\'+str(run_number)+'\\'

    SBN_sim.start_simulation_config(Nx=512,Ny=512,dz=0.05,dims=2, save_dir=saveDir,stride=50)


    #######################################################################################
    #############State Initialization######################################################
    #######################################################################################

    If1=30

    factor_t = SBN_sim.factor_t

    SUPER_GAUSS_EXP = 4

    X, Y = SBN_sim.xx, SBN_sim.yy
    X -= X.max()/2
    Y -= Y.max()/2


    ########################################################
    #################   E1    ##############################
    ########################################################

    velocity = 0

    gaussian_beam = np.zeros(np.shape(X)) + 0j

    wx1 = 600e-6 * factor_t
    wy1 = 600e-6 * factor_t

    x_pos = -0* 200e-6 * factor_t
    y_pos = 0

    gaussian_beam += np.exp(-((2.0* (
                            (X - x_pos) ** 2.0 / (wx1**2.0)
                        +   (Y - y_pos) ** 2.0 / (wy1**2.0)
                            ))** SUPER_GAUSS_EXP)) + 0.00000000000000000001 + 0j

    gaussian_beam*=np.exp(1j*velocity*X)

    print("sound velocity ->", np.sqrt(If1/Isat), "fluid velocity ->", velocity)

    gaussian_beam*=np.sqrt(If1)

    A_noise=0.3
    gaussian_beam = gaussian_beam*(1+whitenoise_2d_field(X,A_noise))
    SBN_sim.field1.add_field_numpy(gaussian_beam)


    ########################################################
    #################   E2    ##############################
    ########################################################

    If2=60

    v2 = 0

    gaussian_beam = np.zeros(np.shape(X)) + 0j

    wx1 = 600e-6 * factor_t
    wy1 = 600e-6 * factor_t

    x_pos = -0* 200e-6 * factor_t
    y_pos = 0

    gaussian_beam += np.exp(-((2.0* (
                            (X - x_pos) ** 2.0 / (wx1**2.0)
                        +   (Y - y_pos) ** 2.0 / (wy1**2.0)
                            ))** SUPER_GAUSS_EXP)) + 0.00000000000000000001 + 0j

    x_pos_vortex = 0
    y_pos_vortex = 0
    theta = np.arctan2(Y-y_pos_vortex,X-x_pos_vortex) 
    l=1
    vortex_phase = np.exp(1j*l*theta)
    gaussian_beam*=vortex_phase

    print("sound velocity", np.sqrt(If1/Isat), velocity)

    gaussian_beam*=np.sqrt(If2)
    A_noise=0.0
    gaussian_beam = gaussian_beam*(1+A_noise*whitenoise_2d_field(X,A_noise))


    SBN_sim.field2.add_field_numpy(gaussian_beam)

    #SBN_sim.plot()

    simulate = True
    if simulate:
        SBN_sim.run()

delta_n -> 0.0001235560064
lx -> 302.4397292938621
ly -> 226.81088976051322
lz -> 58.37032210723811
healing length - > 7.012273216965854e-05
dx-> 0.5907025962770744 dy-> 0.4429900190635024
total steps to simulate -> 23
(512, 512)
(512, 512)
sound velocity -> 0.3872983346207417 fluid velocity -> 0
sound velocity 0.3872983346207417 0
Stride 0 of 23
Stride 1 of 23
Stride 2 of 23
Stride 3 of 23
Stride 4 of 23
Stride 5 of 23
Stride 6 of 23
Stride 7 of 23
Stride 8 of 23
Stride 9 of 23
Stride 10 of 23
Stride 11 of 23
Stride 12 of 23
Stride 13 of 23
Stride 14 of 23
Stride 15 of 23
Stride 16 of 23
Stride 17 of 23
Stride 18 of 23
Stride 19 of 23
Stride 20 of 23
Stride 21 of 23
Stride 22 of 23
delta_n -> 0.0001235560064
lx -> 302.4397292938621
ly -> 226.81088976051322
lz -> 58.37032210723811
healing length - > 7.012273216965854e-05
dx-> 0.5907025962770744 dy-> 0.4429900190635024
total steps to simulate -> 23
(512, 512)
(512, 512)
sound velocity -> 0.3872983346207417 fluid velocity -> 0
sound velo

## 2. Analysis

## 2.1 Load Data

In [16]:
from solver.af_loader import *


sim_state = []
sim_state2 = []

zz= []
n_state=50
for passage in range(0, n_state):
    saveDir = saveDir
    print(saveDir)
    my_mesh = mesh(saveDir+"\\")
    stride_read = 1

    ntotalfiles = len([f for f in os.listdir(saveDir+"\\gnlse_field\\") if f.endswith('.af')])
    index = int(ntotalfiles * passage/n_state)-1

    print(ntotalfiles,passage/n_passages, (index+1)/ntotalfiles)
    
    
    simulation_data,zs = load_data_folder(saveDir+"\\gnlse_field\\", my_mesh,index=1,stride_read=stride_read, index_to_read = index)
    simulation_data2,zs = load_data_folder(saveDir+"\\gnlse_field2\\",my_mesh, index=2, stride_read=stride_read, index_to_read = index)
    
    sim_state.append(np.transpose(simulation_data[0]))
    sim_state2.append(np.transpose(simulation_data2[0]))
    zz.append(zs[0])

    
sim_state=np.array(sim_state)
sim_state2=np.array(sim_state2)

D:\Quarmen2d_sim_new_sims_2passages_256\19\
24 0.0 0.0
D:\Quarmen2d_sim_new_sims_2passages_256\19\
24 0.5 0.0
D:\Quarmen2d_sim_new_sims_2passages_256\19\
24 1.0 0.0
D:\Quarmen2d_sim_new_sims_2passages_256\19\
24 1.5 0.041666666666666664
D:\Quarmen2d_sim_new_sims_2passages_256\19\
24 2.0 0.041666666666666664
D:\Quarmen2d_sim_new_sims_2passages_256\19\
24 2.5 0.08333333333333333
D:\Quarmen2d_sim_new_sims_2passages_256\19\
24 3.0 0.08333333333333333
D:\Quarmen2d_sim_new_sims_2passages_256\19\
24 3.5 0.125
D:\Quarmen2d_sim_new_sims_2passages_256\19\
24 4.0 0.125
D:\Quarmen2d_sim_new_sims_2passages_256\19\
24 4.5 0.16666666666666666
D:\Quarmen2d_sim_new_sims_2passages_256\19\
24 5.0 0.16666666666666666
D:\Quarmen2d_sim_new_sims_2passages_256\19\
24 5.5 0.20833333333333334
D:\Quarmen2d_sim_new_sims_2passages_256\19\
24 6.0 0.20833333333333334
D:\Quarmen2d_sim_new_sims_2passages_256\19\
24 6.5 0.25
D:\Quarmen2d_sim_new_sims_2passages_256\19\
24 7.0 0.25
D:\Quarmen2d_sim_new_sims_2passages_256

## 2.2 Plot States

In [17]:
from solver.analysis_auxiliary_functions import *
import cmocean

index_plot = -1
z_value = zz[index_plot]/SBN_sim.factor_z*1e2
extent = [0,SBN_sim.lx*1e3,0,SBN_sim.ly*1e3]

fig,ax = plt.subplots(1,2,figsize=[10,4])

fig.suptitle(r'$z='+str(np.round(z_value,2))+'$ cm')
ax[0].set_title(r'$|E_1|^2$')
ax[0].imshow(np.abs(sim_state[index_plot])**2, aspect=SBN_sim.Nx / SBN_sim.Ny, extent = extent, origin='lower', cmap=cmocean.cm.speed_r)
ax[0].set_xlabel(r'$x(mm)$')
ax[0].set_ylabel(r'$y(mm)$')

ax[1].set_title(r'$\phi_1$')
ax[1].imshow(np.angle(sim_state[index_plot]), aspect=SBN_sim.Nx / SBN_sim.Ny, extent=extent, origin='lower', cmap=plt.cm.twilight_shifted)
ax[1].set_xlabel(r'$x(mm)$')
ax[1].set_ylabel(r'$y(mm)$')


fig,ax = plt.subplots(1,2,figsize=[10,4])

fig.suptitle(r'$z='+str(np.round(z_value,2))+'$ cm')
ax[0].set_title(r'$|E_2|^2$')
im = ax[0].imshow(np.abs(sim_state2[index_plot])**2, aspect=SBN_sim.Nx / SBN_sim.Ny, extent = extent, origin='lower', cmap=plt.cm.gist_heat)
ax[0].set_xlabel(r'$x(mm)$')
ax[0].set_ylabel(r'$y(mm)$')
plt.colorbar(im)

ax[1].set_title(r'$\phi_2$')
ax[1].imshow(np.angle(sim_state2[index_plot]), aspect=SBN_sim.Nx / SBN_sim.Ny, extent=extent, origin='lower', cmap=plt.cm.twilight_shifted)
ax[1].set_xlabel(r'$x(mm)$')
ax[1].set_ylabel(r'$y(mm)$')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$y(mm)$')

## 3. Track the dynamics of the vortex center

1. Method to compute the center of the vortex
2. Compute the MSD 
3. Try to understand the Heat Bath

In [18]:
fig,ax = plt.subplots(1,1,figsize=[10,4])

extent = [2*SBN_sim.lx*1e3/6,4*SBN_sim.lx*1e3/6,2*SBN_sim.ly*1e3/6,4*SBN_sim.ly*1e3/6]

temp_state = np.abs(sim_state2[index_plot]**2)[int(2*SBN_sim.Ny/6):int(4*SBN_sim.Ny/6),int(2*SBN_sim.Nx/6):int(4*SBN_sim.Nx/6)]
fig.suptitle(r'$z='+str(np.round(z_value,2))+'$ cm')
ax.set_title(r'$|E_2|^2$')
im = ax.imshow(temp_state, aspect=SBN_sim.Nx / SBN_sim.Ny, extent = extent, origin='lower', cmap=plt.cm.gist_heat)
ax.set_xlabel(r'$x(mm)$')
ax.set_ylabel(r'$y(mm)$')
plt.colorbar(im)
xp,yp = np.unravel_index(temp_state.argmin(), temp_state.shape)
print(yp,xp)
ax.plot(extent[0] + xp*SBN_sim.lx*1e3 / SBN_sim.Nx, extent[2] + yp*SBN_sim.ly*1e3 / SBN_sim.Ny ,'o',color='seagreen',ms=10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

85 87


In [30]:
%matplotlib widget
import numpy as np

def compute_velocity_gradients(phi, dx, dy):
    # Calculate the velocity components as gradients of phase
    v_x = np.gradient(np.unwrap(phi,axis=1), axis=1) / dx
    v_y = np.gradient(np.unwrap(phi,axis=0), axis=0) / dy
    return v_x, v_y

def compute_vorticity(v_x, v_y, dx, dy):
    # Compute the vorticity using the difference of partial derivatives of velocity components
    dv_y_dx = np.gradient(v_y, axis=1) / dx
    dv_x_dy = np.gradient(v_x, axis=0) / dy
    vorticity = dv_y_dx - dv_x_dy
    return vorticity

def find_vortices(vorticity, threshold=0.5):
    # Identify points where vorticity is non-zero, indicating potential vortices
    # We use a threshold to account for numerical precision issues
    vortex_positions = np.where(np.abs(vorticity) > threshold)
    return vortex_positions

def main(Phi, dx, dy,threshold=1e-1):
    # Compute velocity gradients from the phase
    v_x, v_y = compute_velocity_gradients(Phi, dx, dy)
    
    # Calculate the vorticity field
    vorticity = compute_vorticity(v_x, v_y, dx, dy)
    
    # Find vortex positions
    vortex_positions = find_vortices(vorticity,threshold=threshold)
    
    return vortex_positions

index_plot=10
temp_state = sim_state2[index_plot][int(2*SBN_sim.Ny/6):int(4*SBN_sim.Ny/6),int(2*SBN_sim.Nx/6):int(4*SBN_sim.Nx/6)]

vortex_positions  = main( np.angle(temp_state), dx=1, dy=1, threshold=1.2)
print(vortex_positions)
print(np.mean(vortex_positions[0]))

fig,ax = plt.subplots(1,1,figsize=[10,4])
fig.suptitle(r'$z='+str(np.round(z_value,2))+'$ cm')
ax.set_title(r'$|E_2|^2$')
im = ax.imshow(np.angle(temp_state), aspect=SBN_sim.Nx / SBN_sim.Ny, extent = extent, origin='lower', cmap=plt.cm.gist_heat)

ax.set_xlabel(r'$x(mm)$')
ax.set_ylabel(r'$y(mm)$')
plt.colorbar(im)
xp,yp = np.unravel_index(temp_state.argmin(), temp_state.shape)
print(yp,xp)
ax.plot(extent[0] + vortex_positions[0]*SBN_sim.lx*1e3 / SBN_sim.Nx, extent[2] + vortex_positions[1]*SBN_sim.ly*1e3 / SBN_sim.Ny ,'o',color='seagreen',ms=10)


(array([85, 85, 86, 86], dtype=int64), array([86, 87, 86, 87], dtype=int64))
85.5


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

101 88


## 3.3 Load For every simulation and compute the vortex position

In [31]:
from solver.af_loader import *
all_x_pos = []
all_y_pos = []

for iii in range(0,20):
    print(iii,' of ', 20, end='\r')

    xpos=[]
    ypos=[]

    sim_state = []
    sim_state2 = []

    zz= []
    n_state=50
    for passage in range(0, n_state):
        saveDir = 'D:\\Quarmen2d_sim_new_sims_2passages_256\\'+str(iii)+"\\"
        my_mesh = mesh(saveDir)
        stride_read = 1

        ntotalfiles = len([f for f in os.listdir(saveDir+"\\gnlse_field\\") if f.endswith('.af')])
        index = int(ntotalfiles * passage/n_state)-1


        
        
        simulation_data,zs = load_data_folder(saveDir+"\\gnlse_field\\", my_mesh,index=1,stride_read=stride_read, index_to_read = index)
        simulation_data2,zs = load_data_folder(saveDir+"\\gnlse_field2\\",my_mesh, index=2, stride_read=stride_read, index_to_read = index)
        
        sim_state.append(np.transpose(simulation_data[0]))
        sim_state2.append(np.transpose(simulation_data2[0]))
        zz.append(zs[0])

        
    sim_state=np.array(sim_state)
    sim_state2=np.array(sim_state2)



    for index_plot in range(5,n_state):
        #extent = [2*SBN_sim.lx*1e3/6,4*SBN_sim.lx*1e3/6,2*SBN_sim.ly*1e3/6,4*SBN_sim.ly*1e3/6]

        temp_state = sim_state2[index_plot][int(2*SBN_sim.Ny/6):int(4*SBN_sim.Ny/6),int(2*SBN_sim.Nx/6):int(4*SBN_sim.Nx/6)]
        #temp_state = np.abs(sim_state2[index_plot]**2)[int(SBN_sim.Ny/4):int(3*SBN_sim.Ny/4),int(SBN_sim.Nx/4):int(3*SBN_sim.Nx/4)]
        

        data = -1*temp_state
        data -= np.min(data)

        h, w = data.shape
        data = data.ravel()


        x = np.linspace(0, w, w)
        y = np.linspace(0, h, h)
        x, y = np.meshgrid(x, y)

        # initial guess of parameters
        
        xp,yp = np.unravel_index(temp_state.argmin(), temp_state.shape)
        
        vortex_positions  = main( np.angle(temp_state), dx=1, dy=1, threshold=0.5)
        xpos1 = np.mean(vortex_positions[0])
        xpos1 = vortex_positions[0][0]
        ypos1 = vortex_positions[1][0]
        ypos1 = np.mean(vortex_positions[1])
        
        # try:
        #     initial_guess = (np.max(data), xp,yp,10,10,0,0)

        #     # find the optimal Gaussian parameters
        #     popt, pcov = curve_fit(twoD_Gaussian, (x, y), data, p0=initial_guess) 
        #     new_xp, new_yp = popt[1],popt[2]
        #     xpos.append(new_xp)
        #     ypos.append(new_yp)
        # except:
        #     print(index_plot)
        #     xpos.append(xp)
        #     ypos.append(yp)

        if np.isnan(xpos1) or np.isnan(ypos1):
            print(ypos1,xpos1, vortex_positions)
            xpos.append(xp)
            ypos.append(yp)
        else:
            xpos.append(xpos1)
            ypos.append(ypos1)


    all_x_pos.append(np.array(xpos))
    all_y_pos.append(np.array(ypos))

all_x_pos = np.array(all_x_pos)
all_y_pos = np.array(all_y_pos)

In [32]:
fig,ax = plt.subplots(figsize=[10,10])

SD_all = []
for ind in range(0,len(all_x_pos)):
    deltax = all_x_pos[ind] - all_x_pos[ind][0]
    deltay = all_y_pos[ind] - all_y_pos[ind][0]
    SD = deltax**2 + deltay**2
    SD_all.append(np.array(SD))
    
    ax.plot(SD,ls='-',lw=0.2,color='k')
SD_all = np.array(SD_all)

ax.plot(np.mean(SD_all,axis=0),color='red')
#ax.plot(np.arange(0,len(SD)),0.05*np.arange(0,len(SD)),label='Brownian Motion Limit')
#ax.plot(np.arange(0,len(SD)),0.0001*np.arange(0,len(SD))**2,label='Superdiffusive Motion')
ax.set_xlabel('z')
ax.set_ylabel('MSD')
#ax.set_ylim(0,2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'MSD')

In [44]:
fig,ax = plt.subplots()
for ind in range(0,len(all_x_pos)):
    deltax = all_x_pos[ind] - all_x_pos[ind][0]
    deltay = all_y_pos[ind] - all_y_pos[ind][0]
    ax.scatter(deltax,deltay,marker = 'o',c=np.arange(0,len(xpos)))
    ax.plot(deltax,deltay,ms=None,ls='--',lw=0.4,c='k')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …